<script async src="https://www.googletagmanager.com/gtag/js?id=UA-59152712-8"></script>
<script>
  window.dataLayer = window.dataLayer || [];
  function gtag(){dataLayer.push(arguments);}
  gtag('js', new Date());

  gtag('config', 'UA-59152712-8');
</script>

# BlackHoles@Home Tutorial: Creating a `BOINC` app using the `WrapperApp`

## Author: Leo Werneck

## This tutorial notebook demonstrates how to write a program that runs in the `BOINC` infrastructure using the `WrapperApp`

## <font color=red>**WARNING**:</font> this tutorial notebook is currently incompatible with Windows

## Introduction:

The [BlackHoles@Home](http://blackholesathome.net/) project allows users to volunteer CPU time so a large number of binary black holes simulations can be performed. The objective is to create a large catalog of [gravitational waveforms](https://en.wikipedia.org/wiki/Gravitational_wave), which can be used by observatories such as [LIGO](https://www.ligo.org), [VIRGO](https://www.virgo-gw.eu), and, in the future, [LISA](https://lisa.nasa.gov) in order to infer what was the source of a detected gravitational wave.

BlackHoles@Home is destined to run on the [BOINC](https://boinc.berkeley.edu) infrastructure (alongside [Einstein@Home](https://einsteinathome.org/) and [many other great projects](https://boinc.berkeley.edu/projects.php)), enabling anyone with a computer to contribute to the construction of the largest numerical relativity gravitational wave catalogs ever produced.

### Additional Reading Material:

* [BOINC's Wiki page](https://boinc.berkeley.edu/trac/wiki)
* [BOINC's WrapperApp Wiki page](https://boinc.berkeley.edu/trac/wiki/WrapperApp)

<a id='toc'></a>

# Table of Contents
$$\label{toc}$$

This tutorial explains how to use the `BOINC` wrapper application to run a simple program. The structture of this notebook is as follows:

1. [Step 1](#introduction): Introduction
1. [Step 2](#compiling_wrapper_app): Compiling the `BOINC` wrapper app for your platform
1. [Step 3](#using_wrapper_app): Using the `BOINC` wrapper app
    1. [Step 3.a](#the_main_application): The main application
    1. [Step 3.b](#compiling_the_main_application): Compiling the main application
    1. [Step 3.c](#job_xml): The `job.xml` file
        1. [Step 3.c.i](#simple_job_xml): A very simple `job.xml` file
        1. [Step 3.c.ii](#job_xml_output_redirect_and_zip): Redirecting and zipping output files
1. [Step 4](#latex_pdf_output): Output this notebook to $\LaTeX$-formatted PDF file

<a id='introduction'></a>

# Step 1: Introduction \[Back to [top](#toc)\]
$$\label{introduction}$$

The [`WrapperApp`](https://boinc.berkeley.edu/trac/wiki/WrapperApp) is the simplest way of converting an existing program into a `BOINC` compatible application. The program that will be actually running is the `WrapperApp` and it will take care of:

* Interfacing with the `BOINC` libraries
* Running the original program
* Handling input/output files

Let us assume a simple `BOINC` application, which is made out of only one program, `bhah_test_app`. The directory of this application should then contain the following files:

* The application file `bhah_test_app` with the name format `appname_version_platform`.
* The `WrapperApp` file with the name format `WrapperAppname_version_platform`.
* The `WrapperApp` configuration file, which we will typically call `appname_version_job.xml`.
* The appication version file, which is called `version.xml`.

<a id='compiling_wrapper_app'></a>

# Step 2: Compiling the `BOINC` wrapper app for your platform \[Back to [top](#toc)\]
$$\label{compiling_wrapper_app}$$

In [1]:
# Step 2: Compiling the BOINC wrapper app
# Step 2.a: Load needed Python modules
import os,sys

# Step 2.b: Add NRPy's root directory to the sys.path()
sys.path.append("..")

# Step 2.c: Load NRPy+'s command line helper module
import cmdline_helper as cmd    # NRPy+: Multi-platform Python command-line interface

# Step 2.d: Set the path to the BOINC source code
path_to_boinc = "~/bhah/boinc"
current_path  = os.getcwd()

# Step 2.e: Check the platform and adjust the compilation command accordingly
if sys.platform == "linux":
    wrapper_compile = "make"

elif sys.platform == "darwin":
    wrapper_compile = "source BuildMacWrapper.sh"
    
else:
    print("Unsupported platform: "+sys.platform)
    sys.exit(1)

# Step 2.f: Compile the wrapper app
!cd $path_to_boinc/samples/wrapper && $wrapper_compile

# Step 2.g: Copy the wrapper app to the current working directory
!cp $path_to_boinc/samples/wrapper/wrapper $current_path


***************************************************
******* Building 64-bit Intel Application *********
***************************************************

BuildMacWrapper.sh:89: no matches found: *.o
rm wrapper *.o
rm: wrapper: No such file or directory
rm: *.o: No such file or directory
make: *** [clean] Error 1
c++  -isysroot /Applications/Xcode.app/Contents/Developer/Platforms/MacOSX.platform/Developer/SDKs/MacOSX11.1.sdk -arch x86_64 -DMAC_OS_X_VERSION_MAX_ALLOWED=1070 -DMAC_OS_X_VERSION_MIN_REQUIRED=1070 -fvisibility=hidden -fvisibility-inlines-hidden -g -I../../clientgui/mac -I../.. -I../../lib -I../../api -I../../zip -stdlib=libc++  -c -o wrapper.o wrapper.cpp
/Applications/Xcode.app/Contents/Developer/usr/bin/gcc -isysroot /Applications/Xcode.app/Contents/Developer/Platforms/MacOSX.platform/Developer/SDKs/MacOSX11.1.sdk -arch x86_64 -DMAC_OS_X_VERSION_MAX_ALLOWED=1070 -DMAC_OS_X_VERSION_MIN_REQUIRED=1070 -fvisibility=hidden -fvisibility-inlines-hidden -g -I../../clientgui/ma


***************************************************
**************** Build Succeeded! *****************
***************************************************



<a id='using_wrapper_app'></a>

# Step 3: Using the `BOINC` wrapper app \[Back to [top](#toc)\]
$$\label{using_wrapper_app}$$

Once we have everything set up, using the wrapper app is as simple as running

```bash
$: ./wrapper
```

The following steps will describe how to set the configuration files so that the wrapper app works as you expect it to.

<a id='the_main_application'></a>

## Step 3.a: The main application \[Back to [top](#toc)\]
$$\label{the_main_application}$$

Let us start by writing a simple application which we will run using the `BOINC` wrapper app. In order for us to be able to see some additional configuration features of the wrapper app, we will make our main application slightly more complicated than a simple "Hello World!" program.

This application takes any number of command line arguments and then prints them to `stdout`, `stderr`, and an output text file.

In [2]:
%%writefile simple_app.c
// Step 0: Load all the necessary C header files
#include <stdio.h>
#include <stdlib.h>

// Program description: this program is just a slightly
//                      more complicated version of the
//                      "Hello World!" program, where
//                      we will be taking some command
//                      line inputs and printing them to
//                      stdout, stderr, and an output file.
int main( int argc, char** argv ) {

  // Step 1: Check correct usage
  if( argc == 1 ) {
    fprintf(stderr,"(ERROR) Correct usage is ./simple_app <command_line_arguments>\n");
    exit(1);
  }

  // Step 2: Print all command line arguments to
  //         stdout, stderr, and an output file
  //
  // Step 2.a: Open the output file
  // Step 2.a.i: Set the output file name
  char filename[100] = "output_file.txt";
  // Step 2.a.ii: Open the file
  FILE* filept = fopen(filename,"w");
  // Step 2.a.iii: Check everything is OK
  if( !filept ) {
    fprintf(stderr,"(ERROR) Could not open file %s\n.",filename);
    exit(1);
  }

  // Step 2.b: Print an information line
  fprintf(stdout,"(INFO) Got the following command line arguments:");
  fprintf(stderr,"(INFO) Got the following command line arguments:");
  fprintf(filept,"(INFO) Got the following command line arguments:");
  
  // Step 2.c: Loop over the command line arguments, printing
  //           them to stdout, stderr, and our output file
  for(int i=1;i<argc;i++) {
    fprintf(stdout," %s",argv[i]);
    fprintf(stderr," %s",argv[i]);
    fprintf(filept," %s",argv[i]);
  }

  // Step 2.d: Add a line break to the output
  fprintf(stdout,"\n");
  fprintf(stderr,"\n");
  fprintf(filept,"\n");

  // Step 2.d: Close the output file
  fclose(filept);

  // All done!
  return 0;
}

Overwriting simple_app.c


<a id='compiling_the_main_application'></a>

## Step 3.b: Compiling the main application \[Back to [top](#toc)\]
$$\label{compiling_the_main_application}$$

We now compile the main application using NRPy+'s `cmdline_helper` module.

In [3]:
cmd.C_compile("simple_app.c","simple_app")

Compiling executable...
(EXEC): Executing `gcc -Ofast -fopenmp -march=native -funroll-loops simple_app.c -o simple_app -lm`...
(BENCH): Finished executing in 0.41248512268066406 seconds.
Finished compilation.


<a id='job_xml'></a>

## Step 3.c: The `job.xml` file \[Back to [top](#toc)\]
$$\label{job_xml}$$

Let's see what happens if we try running the wrapper app:

In [4]:
!rm -f job.xml
cmd.Execute("wrapper")

(EXEC): Executing `./wrapper `...
14:30:17 (1108): wrapper (7.17.26016): starting
14:30:17 (1108): can't open job file job.xml
14:30:17 (1108): can't parse job file: -108
14:30:17 (1108): called boinc_finish(-108)
(BENCH): Finished executing in 2.2458579540252686 seconds.


As can be seen above, the `BOINC` wrapper application requests an input file, `job.xml`, to be present in the current working directory. We will now set up a `job.xml` file for the wrapper app in a way that it works correctly with our `simple_app`. A `job.xml` has the following syntax:

```xml
<job_desc>
    <task>
        ...task_options...
    </task>
    ...additional_options...
</job_desc>
```

All the configurations for the wrapper application are enclosed by the `job_desc` environment. To configure the wrapper to work with our specific application, we provide the `task_options`, while `additional_options` can be provided for additional configuration, as we will see.

<a id='simple_job_xml'></a>

### Step 3.c.i: A very simple `job.xml` file \[Back to [top](#toc)\]
$$\label{simple_job_xml}$$

First, let us start with a very basic configuration: let us ask the wrapper to run our simple application using the command line arguments `1 2 3 4 testing hello world 4 3 2 1`. This is achieved with the following `job.xml` file:

In [5]:
%%writefile job.xml
<job_desc>
    <task>
      <application>simple_app</application>
      <command_line>1 2 3 4 testing hello world 4 3 2 1</command_line>
    </task>
</job_desc>

Writing job.xml


Let us now copy everything into a new, fresh directory and run our wrapper application.

In [6]:
!rm -rf wrapper_app_test
cmd.mkdir("wrapper_app_test")
!cp wrapper simple_app job.xml wrapper_app_test && cd wrapper_app_test && ./wrapper && ls

14:30:23 (1113): wrapper (7.17.26016): starting
(INFO) Got the following command line arguments: 1 2 3 4 testing hello world 4 3 2 1
boinc_finish_called    simple_app             wrapper_checkpoint.txt
job.xml                stderr.txt
output_file.txt        wrapper


Note that after execution, we see the output `(INFO) Got the following command line arguments: 1 2 3 4 testing hello world 4 3 2 1` printed to `stdout`. If we examine the file `output_file.txt`, we will see the same output:

In [7]:
!cat output_file.txt

(INFO) Got the following command line arguments: 1 2 3 4 testing hello world 4 3 2 1


The `stderr.txt` file is automatically generated by the `BOINC` wrapper app, and it contains all the output which was sent to `stderr`. We see that while we also have the expected output in it, there is also some additional information which was generated by the wrapper app:

In [8]:
!cat stderr.txt

13:02:19 (92249): Can't open init data file - running in standalone mode
13:02:19 (92249): called boinc_finish(0)
14:00:39 (97771): Can't open init data file - running in standalone mode
14:00:39 (97771): wrapper (7.17.26016): starting
14:00:39 (97771): wrapper: running simple_app (1 2 3 4 testing hello world 4 3 2 1)
(INFO) Got the following command line arguments: 1 2 3 4 testing hello world 4 3 2 1
14:00:40 (97771): simple_app exited; CPU time 0.017981
14:00:40 (97771): called boinc_finish(0)
14:25:38 (420): Can't open init data file - running in standalone mode
14:25:38 (420): wrapper (7.17.26016): starting


Aditionally, we see that the wrapper application has created two additional files: the `boinc_finish_called` and the `wrapper_checkpoint.txt`. For our purposes, the `wrapper_checkpoint.txt` file is irrelevant, so we will ignore it for now. The `boinc_finish_called` file contains the numerical value returned by our program, `simple_app`. As is usual in `C`, if the return value is `0`, then the execution was successful, while a non-zero value indicates an error:

In [9]:
!cat boinc_finish_called

0


<a id='job_xml_output_redirect_and_zip'></a>

### Step 3.c.ii: Redirecting and zipping output files  \[Back to [top](#toc)\]
$$\label{job_xml_output_redirect_and_zip}$$

Now that we have seen the simplest possible case, let us look at something slightly more complicated. The following `job.xml` file asks the wrapper app to perform the following tasks:

1. Run the `simple_app` application with command line arguments `1 2 3 4 testing hello world 4 3 2 1`
1. Redirect all `stdout` output to the file `simple_app.out`
1. Redirect all `stderr` output to the file `simple_app.err`
1. Zip all the output files we have seen before into a single file: `output.zip`

In [10]:
%%writefile job.xml
<job_desc>
    <task>
      <application>simple_app</application>
      <command_line>1 2 3 4 testing hello world 4 3 2 1</command_line>
      <stdout_filename>simple_app.out</stdout_filename>
      <stderr_filename>simple_app.err</stderr_filename>
    </task>
    <zip_output>
      <zipfilename>output.zip</zipfilename>
      <filename>simple_app.out</filename>
      <filename>simple_app.err</filename>
      <filename>output_file.txt</filename>
      <filename>boinc_finish_called</filename>
      <filename>wrapper_checkpoint.txt</filename>
    </zip_output>
</job_desc>

Overwriting job.xml


Now let us see what happens when we run the wrapper app:

In [11]:
!rm -rf wrapper_app_test
cmd.mkdir("wrapper_app_test")
!cp wrapper simple_app job.xml wrapper_app_test && cd wrapper_app_test && ./wrapper && ls

14:30:27 (1123): wrapper (7.17.26016): starting
boinc_finish_called    output_file.txt        stderr.txt
initial_file_list      simple_app             wrapper
job.xml                simple_app.err         wrapper_checkpoint.txt
output.zip             simple_app.out


Notice that we now have the output files `simple_app.out` and `simple_app.err`, as expected. The file `stderr.txt` is still present, by default. We also have all our output files neatly collected into a single zip file, `output.zip`. Note that zipping the output is not done with the goal of reducing the overall size of the output, but because it easier to communicate the output files back to the `BOINC` server.

<a id='latex_pdf_output'></a>

# Step 4: Output this notebook to $\LaTeX$-formatted PDF file \[Back to [top](#toc)\]
$$\label{latex_pdf_output}$$

The following code cell converts this Jupyter notebook into a proper, clickable $\LaTeX$-formatted PDF file. After the cell is successfully run, the generated PDF may be found in the root NRPy+ tutorial directory, with filename
[Tutorial-BlackHolesAtHome-Creating_a_BOINC_app_using_the_WrapperApp.pdf](Tutorial-BlackHolesAtHome-Creating_a_BOINC_app_using_the_WrapperApp.pdf) (Note that clicking on this link may not work; you may need to open the PDF file through another means.)

In [12]:
!cp ../latex_nrpy_style.tplx .
cmd.output_Jupyter_notebook_to_LaTeXed_PDF("Tutorial-BOINC_docker_server_setup")
!rm -f latex_nrpy_style.tplx

[NbConvertApp] WARNING | pattern 'Tutorial-BOINC_docker_server_setup.ipynb' matched no files
Created Tutorial-BOINC_docker_server_setup.tex, and compiled LaTeX file to
    PDF file Tutorial-BOINC_docker_server_setup.pdf
